## Import Neo4j

In [1]:
from neo4j import GraphDatabase

ModuleNotFoundError: No module named 'neo4j'

In [ ]:
import pandas as pd

#### Create Connection to Neo4j Database

In [ ]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [ ]:
conn = Neo4jConnection(uri="neo4j://127.0.0.1:7687", user="team", pwd="F0110wTh€M0n€y")

#### Pseudo-Code

Flagge alle Terror-Transaktionen mit Impurity = 1.
Berechne für alle Adressem, die als Output rauskommen die gewichteten Inputs und Outputs, um für jeden Zeitpunkt die Purity zu erhalten.
Ordne alle Terror-Transaktionen und mach dann die Brechnung von hinten nach vorne (alt zu neu) 
Definiere Abbruchkriterium, wann man die Impurity als Null ansieht (z.B. auf zwei Nachkommastellen beschränken)
Input: Adresse a mit Impurity
for transaction where a SENDS:
    Create list of output addresses
    create DataFrame Column with amount of BTC 
    Create DataFrame Column with Purity of received BTC
    Create DataFrame Column with total BTC of Address at point in time of transaction
    Calculate Purity for address at that point in time

Wir setzen nicht alle Impurity auf 0, sondern interpretieren None später als 0, falls dies nicht geht, dann doch alles auf 0 setzen. Aber muss in Batches gemacht werden, ansonsten crasht neo4j

In [ ]:
#flagImpurityTx = """MATCH (tx:Transaction)
#SET tx.impurity = 0
#RETURN tx.impurity
#LIMIT 10"""

In [ ]:
#conn.query(flagImpurityTx, db='neo4j')

#### Schritt 1: Setze für alle Terror-Adressen und Transaktionen mit Terror-Input Impurity = 1

In [25]:
flagImpurityTerror = """MATCH (terror:Address{isTerrorMultiInput:TRUE})-[SENDS]->(tx:Transaction)
SET tx.impurity = 1
RETURN tx.impurity
LIMIT 10"""

In [34]:
conn.query(flagImpurityTerror, db='neo4j')

#### Schritt 2: Impurity von Terror-Adressen immer als 1 annehmen - Und rechne den Change-Output raus. Das mit den Change-Outputs lassen wir erstmal weg.

#### Schritt 3: Gib alle Output-Adressen aus Terror-Transaktionen aus

In [ ]:
TerrorTxOutputQuery = """MATCH (tx1:Transaction{impurity:1})-[RECEIVES]->(a:Address)-[SENDS]->(tx2:Transaction)
WHERE tx2 <> tx1
RETURN tx2.date AS Date, tx2.txid AS TxID
LIMIT 10"""

In [6]:
response = conn.query(TerrorTxOutputQuery, db='neo4j')

In [24]:
TerrorTxOutput = list(map(lambda x : dict(x), response))
TransactionsDF = pd.DataFrame(TerrorTxOutputAddresses)
TransactionsDF = TransactionsDF[['TxID']].drop_duplicates()
TransactionsDF = TransactionsDF.sort_values(by = ['Date'], ascending = True)

In [ ]:
TransactionsDF.to_csv("./TransactionsDF.csv")

#### Schritt 4: Berechne in chronologischer Reihenfolge für alle Transaktionen die Impurity. Beachte dabei das Datum, und dass ältere Transaktionen wiederum die Impurity neuerer Transaktionen beeinflussen können. Das geht doch nicht, da sich die Impurity-Werte von Adressen ändern, wenn sie abwechseln Inputs und Outputs haben. Deshalb hängen wir die Impurity-Werte and die Sends-Kanten und an die Transaktionen/Receive-Kanten. WIr benutzen die Receives Kanten, damit wir später noch den Change-Output rausrechnen können, wenn wir noch Zeit haben.

In [ ]:
%(transaction_id, address, transaction_date, address, transaction_date, transaction_id)
for row in TransactionsDF(): #Endergebnis: set purity for all recieve edges of transaction
# Find all input addresses to a given transaction
    transaction_id = TransactionsDF[['TxID']]
    transaction_date = TransactionsDF[['Date']]
    #tx:'%s' is the transaction_id of the original transaction where we want to determine and set the purity 
    AddressQuery = """MATCH (a:Address)-[SENDS]->(tx:'%s')
    RETURN a.address AS Address"""
    response = conn.query(AddressQuery, db='neo4j')
    AddressesTransaction = list(map(lambda x : dict(x), response))
    # Find all input transactions for a given address with the amount of Bitcoin and impurity value of the transaction up to the date
    for i in AddressesTransaction():
        address = AddressesTransaction[i]
        #return for each recieving edge of address the recieves_value and recieves_purity
        #return for each send edge the send_value and send_purity
        #'%s' is the address of AddressesTransaction
        #'%s' is the transaction_date of the original transaction
        RecieviesOfAddress = """MATCH (tx:Transaction)-[rec:RECEIVES]->(a:Address{address:'%s'})
        WHERE tx.date > '%s'
        RETURN rec.value AS Received_Value, rec.purity AS Received_Purity
        """
        response = conn.query(RecieviesOfAddress, db='neo4j')
        ReceivesOfAddress = list(map(lambda x : dict(x), response))
        ReceivesOfAddress = pd.DataFrame(ReceivesOfAddress)
        
        #'%s' is the address of AddressesTransaction
        #'%s' is the transaction_date of the original transaction
        #'%s' is the transacrion_id of the original transaction
        SendsOfAddress"""MATCH (a:Address{address:'%s'})-[send:SENDS]->(tx:Transaction)
        WHERE tx.date > '%s', tx.txid <> '%s'
        RETURN send.value AS Send_Value, send.purity as Send_Purity
        """
        response = conn.query(SendsOfAddress, db='neo4j')
        SendsOfAddress = list(map(lambda x : dict(x), response))
        SendsOfAddress = pd.DataFrame(SendsOfAddress)
        
        #calculate inpurity of address
        
            
        
        #Endergebnis: set purity of send edge from address to transaction

#for trial purposes: 
#"""MATCH (tx:Transaction)-[rec:RECEIVES]->(a:Address)
#WHERE tx.date > date({year: 2019, month: 7, day: 1})
#RETURN rec.value AS Received_Value, rec.purity AS Received_Purity,  tx.date AS txDate
#LIMIT 10"""
#"""MATCH p=((a:Address{address:"15PLEMoh11HBiHf53d4UCfJdx9ALyfnCT3"})-[send:SENDS]->(tx:Transaction))
#WHERE tx.date > date({year: 2016, month: 7, day: 1})
#RETURN p"""
 



In [13]:
dataframeExampleQuery = """MATCH (tx:Transaction)-[rec:RECEIVES]->(a:Address{address:"15PLEMoh11HBiHf53d4UCfJdx9ALyfnCT3"})
WHERE tx.date > date({year: 2019, month: 7, day: 1})
RETURN rec.value AS Received_Value, rec.purity AS Received_Purity,  tx.date AS txDate
LIMIT 10"""

response = conn.query(dataframeExampleQuery, db='neo4j')
dataframeExample = list(map(lambda x : dict(x), response))
ExampleDF = pd.DataFrame(dataframeExample)

AttributeError: 'list' object has no attribute 'records'

In [ ]:

for transaction in TransactionsDF:
    response = conn.query(TerrorTxOutputQuery, db='neo4j')
    

#### Schritt 6: Gib alle Output-Adressen aus den betrachteten Transaktionen aus und berechne für alle Nicht-Terror-Adressen in chronologischer Reihenfolge die Impurity der Transaktionen

#### Schritt 7: Wiederhole Schritte 5 und 6 bis du auf einen Exchange triffst

#### Query for all transaction with a terror-input (Set Impurity = 1)

In [ ]:
"""MATCH (terror:Address{isTerror:TRUE})-[SENDS]->(tx:Transaction)
SET tx.TerrorInput = TRUE
RETURN tx
Limit 10"""

In [ ]:
def ImpurityMeasure(Address, Date):
    Transaction_Data = [Date, TX ID, BTC, TX_Impurity]
    Transaction_Data['Total TX Value'] = Transaction_Data['BTC'].cumsum()
    Transaction_Data['Address Impurity'] = Transaction_Data['BTC']*Transaction_Data['TX_Impurity']/Transaction_Data['Total TX Value']
    Impurity_Date = Transaction_Data[Transaction_Data['Date'] < Date].tail(1)
    return Impurity_Date

In [ ]:
AddressQuery = """MATCH (terror:Address{isTerror:TRUE})-[SENDS]->(tx:Transaction)-[RECEIVES]->(a:Address)
   WHERE a.isTerrorMultiInput<> TRUE
   RETURN a.address AS Address"""

In [ ]:
Addresses = AddressQuery()

In [ ]:
for Address in Addresses:
    